In [2]:
import utils.datasaur as data

/home/danim/HoardingDisorderScripts/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-08-28 16:15:15 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2025-08-28 16:15:15 INFO: Downloaded file to /home/danim/stanza_resources/resources.json
2025-08-28 16:15:15 WARNING: Language en package default expects mwt, which has been added
2025-08-28 16:15:15 INFO: Loading these models for language: en (English):
| Processor    | Package             |
--------------------------------------
| tokenize     | combined            |
| mwt          | combined            |
| pos          | combined_charlm     |
| constituency | ptb3-revised_charlm |

2025-08-28 16:15:17 

In [3]:
%load_ext autoreload
%autoreload 2

# Misspellings/Wrong Labels

First thing we're going to do is deal with the issue of misspellings. 
Inspired by Tava's function in `regexes.py`, we're going to use `get_close_matches` to look for words that are close to 

In [4]:
doc_token_content = [
    (doc, [
        (doc.row_data[i]["tokens"], i) 
        for i in range(len(doc.row_data))
    ])
    for doc in data.by_doc 
]

In [5]:
from difflib import get_close_matches
import utils.regexes as rxs
import string
import re

threshold = 0.85
BLACKLIST = ['Interview', 'Participants', 'Interviewees', 'Interviewer 1', 
             'Interviewer 2', 'Participate', '"Participant',
             'Broparticipant', 'Taparticipant'] # ??? Found these have no idea what happened here
potential_spelling_issues = [
    {
        "broken_label" : token_fixed, # fixed due to processing, broken due to misspelling or other error
        "close_to" : matches,
        "doc" : doc,
        "row_num" : row_num,
        "prev_line" : [doc.lines[row_num-1]],
        "line" : doc.lines[row_num]
    }
    for doc, token_list in doc_token_content
    for tokenized_line, row_num in token_list
    for token in tokenized_line
    if (matches := get_close_matches(token, rxs.SPEAKERS, 
                                     cutoff=threshold))
    and (token_fixed := token.title().rstrip(string.punctuation)) 
        not in BLACKLIST
    and not any(token_fixed == match for match in matches)
]
potential_spelling_issues

[{'broken_label': 'Speaker1',
  'close_to': ['Speaker'],
  'doc': Document(3001_039.txt, s1046-50_s2012-13_s3026-50),
  'row_num': 18,
  'prev_line': ['Are there any pictures that come to mind when you think of those words or CBT?'],
  'line': 'Speaker1 – To me just from being in therapy so much all the time with the children, I think of holding hands or a hand reaching out to grab another hand because it’s like your giving a tool, a way of lie, a way to combat something that can be overwhelming, so somebody’s helping you, helping hand to lift you up and get you to that next step.'},
 {'broken_label': '1:Interviewer',
  'close_to': ['Interviewer'],
  'doc': Document(3001_076.txt, s1062_s2022-26_s3076-97),
  'row_num': 9,
  'prev_line': ['Speaker: In my mind I almost feel like you can do both in one but it would probably need to be longer than three minutes. I don’t know how your content would go but I feel like a kids one would be a little different than what a parent’s one would be. I

In [6]:
# There is no issue that is close to more than one speaker label
not any(len(item["close_to"]) != 1 for item in potential_spelling_issues)

True

In [24]:
# Find all instances of this one particular issue
all_doc_content_lol = '\n'.join(doc.full_content for doc in data.by_doc)
weird_number_in_front = re.findall(r'\d:[a-zA-Z]+', all_doc_content_lol)
weird_number_in_front

['1:Interviewer',
 '1:Speaker',
 '1:Speaker',
 '1:Interviewer',
 '1:Speaker',
 '1:Speaker',
 '1:Interviewer',
 '1:Speaker',
 '1:Interviewer',
 '1:Note',
 '1:Interviewer',
 '1:Interviewer',
 '1:Speaker',
 '1:Interviewer',
 '1:Speaker',
 '1:Interviewer',
 '1:Speaker',
 '1:Interviewer',
 '1:Speaker',
 '1:Interviewer',
 '1:Speaker',
 '1:Interviewer',
 '1:Speaker',
 '1:Interviewer',
 '1:Speaker',
 '1:Interviewer',
 '1:Speaker',
 '1:Interviewer']

Fixing this:

In [26]:
list(map(lambda s : s.replace('1:', ''), weird_number_in_front))

['Interviewer',
 'Speaker',
 'Speaker',
 'Interviewer',
 'Speaker',
 'Speaker',
 'Interviewer',
 'Speaker',
 'Interviewer',
 'Note',
 'Interviewer',
 'Interviewer',
 'Speaker',
 'Interviewer',
 'Speaker',
 'Interviewer',
 'Speaker',
 'Interviewer',
 'Speaker',
 'Interviewer',
 'Speaker',
 'Interviewer',
 'Speaker',
 'Interviewer',
 'Speaker',
 'Interviewer',
 'Speaker',
 'Interviewer']

In [27]:
# And these ones
number_issues = []
prepended = re.findall(r'\d(?:{sl})'.format(sl='|'.join(rxs.SPEAKERS)), 
                       all_doc_content_lol)
appended = re.findall(r'(?:{sl})\d'.format(sl='|'.join(rxs.SPEAKERS)), 
                      all_doc_content_lol)
number_issues = list(set(prepended) | set(appended))
number_issues

['2Participant', 'Speaker1', 'Interviewer1']

In [ ]:
the_rest = [
    token
    for item in potential_spelling_issues
    if (token := item["broken_label"]) not in 
    weird_number_in_front + number_issues
]
the_rest

['Spaker', 'Speake', '[Participant']

In [ ]:
[item
 for item in potential_spelling_issues
 if item["broken_label"] == '[Participant']

[{'broken_label': '[Participant',
  'close_to': ['Participant'],
  'doc': Document(040_487.txt, s1036-42_s2008-9_s3000-15),
  'row_num': 26,
  'prev_line': ['Wanted to put it right handy in a little container.'],
  'line': 'I wanted to put it up with the salt and the oatmeal, and my husband said [inaudible 00:45:33] and he harrumphed and " [Participant 040 00:22:36]?" And I looked at what I had and what I was doing and promptly put it where it was supposed to be.'}]

In [ ]:
# don't care about the last one
the_rest = the_rest[:-1]
the_rest # only misspellings left

['Spaker', 'Speake']

All this led to `fix_misspelled_labels.py`.

# Fix spacing issues